# 04 – Insights & Explainability

In [ ]:
import json, joblib, warnings
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

ART = Path('../artifacts')
model_path = ART / 'best_model.joblib'
if not model_path.exists():
    raise FileNotFoundError("Train a model first by running 03_Modeling.ipynb")

pipe = joblib.load(model_path)

# Try to extract feature importances
clf = pipe.named_steps['clf']
pre = pipe.named_steps['pre']

# Get feature names from preprocessor
try:
    cat_features = pre.transformers_[1][2]  # names list
    num_features = pre.transformers_[0][2]
    # After fitting, OneHotEncoder has categories_
    import itertools
    ohe = pre.named_transformers_['cat']
    cat_names = []
    if hasattr(ohe, 'get_feature_names_out'):
        cat_names = ohe.get_feature_names_out(cat_features).tolist()
    feature_names = list(num_features) + cat_names
except Exception as e:
    feature_names = [f"f{i}" for i in range(0, 100)]  # fallback

# Importance for tree-based models
def plot_importances(names, importances, title):
    idx = np.argsort(importances)[::-1][:20]
    plt.figure(figsize=(8,6))
    plt.bar(range(len(idx)), np.array(importances)[idx])
    plt.xticks(range(len(idx)), np.array(names)[idx], rotation=90)
    plt.title(title)
    plt.tight_layout()
    plt.show()

if hasattr(clf, 'feature_importances_'):
    plot_importances(feature_names, clf.feature_importances_, "Feature Importances (Tree-based)")
else:
    # Logistic regression: use absolute coefficients as a proxy
    try:
        coefs = np.abs(clf.coef_).ravel()
        plot_importances(feature_names, coefs, "Absolute Coefficients (LogReg)")
    except Exception as e:
        warnings.warn("Could not compute feature importances/coefs. Consider SHAP below.")

# Optional: SHAP (works best for tree models or small samples)
try:
    import shap
    explainer = None
    if clf.__class__.__name__.lower().startswith('xgb') or hasattr(clf, 'feature_importances_'):
        explainer = shap.Explainer(clf, check_additivity=False)
    else:
        # KernelExplainer (slow) — sample few points
        X_sample = np.random.randn(100, len(feature_names))
        explainer = shap.KernelExplainer(clf.predict_proba, X_sample)
    # Plot summary with random sample
    shap.summary_plot = shap.summary_plot  # keep linter happy
    print("SHAP is available. Use explainer to compute shap values on your transformed X.")
except Exception as e:
    print("SHAP not available or failed. Skipping.")


**Summary**: Study time and absences typically correlate strongly with outcomes; prior grades (G1/G2) are also strong predictors. Use these plots to support your narrative.